This an idea on how to use lat lon data to create a new variable. At first I wanted to use NYC's districts polygon map to derive a district appartenance for each listing_id but that would have been using external data. So instead of this I figured I could just derive natural district from the lat long data using a little clustering. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
train=pd.read_json("../input/train.json")
test=pd.read_json("../input/test.json")
train["Source"]='train'
test["Source"]='test'
data=pd.concat([train, test]) 

In [ ]:
plt.scatter(data["longitude"], data["latitude"], s=5)
plt.title("Geographical positions of the listings")
plt.show()

## Geographic plotting ##

Clearly, it seems that some of the data is missing (0,0): unless there are some people are looking for an appartment right in the guinea gulf.  Let's remove them so we can have a better view of the data. 

In [ ]:
plt.scatter(data.loc[data["longitude"]<-60,"longitude"], data.loc[data["latitude"]>20,"latitude"], s=5)
plt.title("Geographical positions of the listings")
plt.show()

There are a few flats all around the US but most of the cloud is around NYC
So lets zoom in on NYC 

In [ ]:
plt.scatter(data.loc[(data["longitude"]<-73.75)&(data["longitude"]>-74.05)&(data["latitude"]>40.4)&(data["latitude"]<40.9),"longitude"],
                      data.loc[(data["latitude"]>40.4)&(data["latitude"]<40.9)&(data["longitude"]<-73.75)&(data["longitude"]>-74.05),"latitude"], s=5)
plt.title("Geographical positions of the listings")
plt.show()

At this level there are enough points to see some known features: we see the shape of manhattan with a hole for central park for example. 

## Clustering NYC data ##

In [ ]:


#I use Birch because of how fast it is. 
from sklearn.cluster import Birch
def cluster_latlon(n_clusters, data):  
    #split the data between "around NYC" and "other locations" basically our first two clusters 
    data_c=data[(data.longitude>-74.05)&(data.longitude<-73.75)&(data.latitude>40.4)&(data.latitude<40.9)]
    data_e=data[~((data.longitude>-74.05)&(data.longitude<-73.75)&(data.latitude>40.4)&(data.latitude<40.9))]
    #put it in matrix form
    coords=data_c.as_matrix(columns=['latitude', "longitude"])
    
    brc = Birch(branching_factor=100, n_clusters=n_clusters, threshold=0.01,compute_labels=True)

    brc.fit(coords)
    clusters=brc.predict(coords)
    data_c["cluster_"+str(n_clusters)]=clusters
    data_e["cluster_"+str(n_clusters)]=-1 #assign cluster label -1 for the non NYC listings 
    data=pd.concat([data_c,data_e])
    plt.scatter(data_c["longitude"], data_c["latitude"], c=data_c["cluster_"+str(n_clusters)], s=10, linewidth=0.1)
    plt.title(str(n_clusters)+" Neighbourhoods from clustering")
    plt.show()
    return data 

In [ ]:
cluster_latlon(50, data)

The ideal algorithm for this would be DBSCAN as shown in here http://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/ however it is too heavy to run on this kernel because I believe it has to compute a matrix of distance between each points. The advantage of DBSCAN is that it would leave the "extremum" point out of the clusters while Birch here is creating clusters of very low density for those 

## Naive variables##

In [ ]:
data["created"]=pd.to_datetime(data["created"])
data["created_month"]=data["created"].dt.month
data["created_day"]=data["created"].dt.day
data["created_hour"]=data["created"].dt.hour

In [ ]:
data["num_photos"]=data["photos"].apply(len)
data["num_features"]=data["features"].apply(len)
data["num_description_words"] = data["description"].apply(lambda x: len(x.split(" ")))

In [ ]:
features_to_use  = ["bathrooms", "bedrooms", "price", 
                                                     
                    "num_photos", "num_features", "num_description_words",                    
                    "created_month", "created_day", "created_hour"
                   ]

In [ ]:
train=data[data["Source"]=="train"]
test=data[data["Source"]=="test"]
target_num_map={"high":0, "medium":1, "low":2}
y=np.array(train["interest_level"].apply(lambda x: target_num_map[x]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val,y_train, y_val =train_test_split( train[features_to_use], y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
clf=RFC(n_estimators=1000, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import log_loss
y_val_pred = clf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

## Adding the cluster variable ##

In [ ]:
from sklearn.metrics import log_loss


def compute_logloss(n_cluster,data):
    data_cluster=cluster_latlon(n_cluster,data)
    train=data_cluster[data_cluster["Source"]=="train"]

    target_num_map={"high":0, "medium":1, "low":2}
    y=np.array(train["interest_level"].apply(lambda x: target_num_map[x]))
    
    features = ["bathrooms", "bedrooms", "price", 
                                                        
                    "num_photos", "num_features", "num_description_words",                    
                    "created_month", "created_day", "created_hour", "cluster_"+str(n_cluster)
                   ]
    
    X_train, X_val,y_train, y_val =train_test_split( train[features], y, test_size=0.33, random_state=42)
    clf.fit(X_train, y_train)

    y_val_pred = clf.predict_proba(X_val)
    return log_loss(y_val, y_val_pred)

In [ ]:
compute_logloss(3, data)

A tiny bit better but lets check with more clusters

In [ ]:
log_loss_cls={}
for n in range(4,15):
    log_loss_cls[n]=compute_logloss(n, data)
    
n_c = sorted(log_loss_cls.items()) 
x, y = zip(*n_c) 
plt.plot(x, y)
plt.title("log_loss for different numbers of clusters")
plt.show()

In [ ]:
log_loss_cls

It seems that the more clusters, the better the log_loss becomes. A the extreme of this each point is his own cluster and we are back to the lat ,lon original data